# Scraper for Past Seasons from CHN

## For use on seasons 2021 and before that don't have advanced metrics

In [ ]:
# Dependencies
## Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import logging
from tqdm import tqdm
import re
from sqlalchemy import create_engine
import sqlite3


### SET TARGET YEAR
season_year = 2016

# Tags and Paths
FILE_TAG = f'{season_year}_Box_Scores_v1'

## global variables
current_year_url = f'https://www.collegehockeynews.com/schedules/?season={season_year}{season_year+1}'

tag = FILE_TAG
db_name = FILE_TAG
result_table_name = season_year


## Base usl for box scores and metrics

base_url = 'https://www.collegehockeynews.com'

### Download a table of every game in CHN database for the selected season

In [ ]:
## Functions
### Parse the current season schedule / results page

def parse_current_season(url):
        # Initialize variables
    current_date = None
    current_conference = None
    game_notes = None

    # Initialize an empty list to hold the data
    data = []

    # Parse the page with BeautifulSoup
    # Get the page with requests
    response = requests.get(url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # select the table or tables
    tables = soup.find_all('table')

    rows = soup.find_all('tr')

    # Loop through each row to find relevant information
    for row in rows:
        # Check for date row
        if row.get('class') == ['stats-section']:
            current_date = row.find('td').text.strip()
        # Check for conference row
        elif row.get('class') == ['sked-header']:
            current_conference = row.find('td').text.strip()
        # Check for game notes
        elif len(row.find_all('td')) == 2:
            game_notes = row.find_all('td')[1].text.strip()
        # Process rows with game data
        elif row.get('valign') == 'top':
            cells = row.find_all('td')
            if len(cells) >= 9:
                home_team = cells[0].text.strip()
                # Remove any hyphens from the team name
                home_team = home_team.replace('-', ' ')
                home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
                home_score = cells[1].text.strip()
                away_team = cells[3].text.strip()
                away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
                away_score = cells[4].text.strip()
                ot = cells[5].text.strip()
                box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
                metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
                # Capture Game Notes
                game_notes_cell = cells[-1].find('small')
                game_notes = game_notes_cell.text.strip() if game_notes_cell else None

                # Append data to the list
                data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
                game_notes = None  # Reset game notes for the next row
    return data

## call the function
data = parse_current_season(current_year_url)


# Create a dataframe from the list

columns = ['Date', 'Conference', 'Game_Notes', 'Home_Team', 'Home_Team_Link', 'Home_Score', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)
            
## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# remove day of the week from date
# format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

### Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    # Remove any hyphens from the team name if there are any
    home_team_abbr = home_team_abbr.replace('-', ' ')
    away_team_abbr = away_team_abbr.replace('-', ' ')
    game_id = f'{row.Date}-{home_team_abbr}-{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Remove any hyphens from the team names if any
df['Home_Team'] = df['Home_Team'].str.replace('-', ' ')
df['Away_Team'] = df['Away_Team'].str.replace('-', ' ')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}-{row.Home_Team}-{row.Away_Team}', axis=1)

## Filter out games that have not been played yet
df = df[df['Home_Score'] != '']

# Replace Nan values in metrics column with empty string
df['Metrics_Link'] = df['Metrics_Link'].fillna('')

# save the dataframe to a csv file for manual review
df.to_csv('../TEMP/season_table_test.csv')

# Save copy of df before filter
df_pre = df.copy()

# Filter out Exhibition games
df = df[~df['Conference'].str.contains('Exhibition')]


# Print length of DF before and after filter
print(len(data))
print(len(df_pre))
print(len(df))


# Sub Functions

In [ ]:
## Functions for parsing the box score and metrics pages

# Initialize logging for Error and Warning messages
logging.basicConfig(filename='../TEMP/PAST_scrape.log', level=logging.INFO)

#### PARSE PLAYER STATS TABLE ####
def parse_player_summary(html_content):
    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the playersums div
    playersums_div = soup.find('div', id='playersums')
    if playersums_div is None:
        return "Player summaries div not found"

    # Initialize list to store player stats
    player_stats = []

    # Loop through each playersum div
    for player_sum in playersums_div.find_all('div', class_='playersum'):
        team = player_sum.find('td').text.strip()
        
        # Loop through table rows
        for row in player_sum.find_all('tr'):
            cols = row.find_all('td')
            if len(cols) > 1:
                player = cols[0].text.strip()
                goals = cols[1].text.strip()
                assists = cols[2].text.strip()
                points = cols[3].text.strip()
                plus_minus = cols[4].text.strip()
                shots = cols[5].text.strip()
                pim = cols[6].text.strip()
                fowl = cols[7].text.strip() if len(cols) > 7 else None
                
                fow, fol = None, None
                win_percentage = None
                
                

                try:
                    if fowl and '‑' in fowl:  # Checking if it contains a hyphen
                        fow, fol = map(int, fowl.split('‑'))
                        total_fo = fow + fol
                        win_percentage = (fow / total_fo) * 100 if total_fo > 0 else 0
                except ValueError:
                    fow, fol, win_percentage = None, None, None

                

                
                player_stat = {
                    'Team': team,
                    'Player': player,
                    'G': goals,
                    'A': assists,
                    'Pt.': points,
                    '+/-': plus_minus,
                    'Sh': shots,
                    'PIM': pim,
                    'FOW': fow,
                    'FOL': fol,
                    'FO%': win_percentage
                }
                player_stats.append(player_stat)

    return pd.DataFrame(player_stats)


############# PARSEING SCORING SUMMARY WITH BS4

def parse_scoring_summary(html_content):
    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the scoring div and table
    scoring_div = soup.find('div', id='scoring')
    if scoring_div is None:
        logging.error("Scoring div not found")
        return None

    scoring_table = scoring_div.find('table')
    if scoring_table is None:
        logging.error("Scoring table not found within the scoring div")
        return None

    # Initialize list to store scoring events
    scoring_events = []
    period = None

    # Loop through table rows
    for row in scoring_table.find_all('tr'):
        if 'stats-section' in row.get('class', []):
            td = row.find('td')
            if td:
                period = td.text.strip()
            else:
                logging.warning("Period name not found in 'stats-section' row")
                period = "Unknown"
        else:
            cols = row.find_all('td')
            if len(cols) > 1:
                try:
                    team = cols[0].text.strip()
                    pp = cols[1].text.strip()

                    player_data = cols[3].text.strip()
                    match = re.match(r"(.+)\s\((\d+)\)", player_data)
                    player = match.group(1) if match else player_data
                    goals = int(match.group(2)) if match else None

                    assist_data_raw = cols[4].text.strip()
                    assist_data = assist_data_raw.split(", ") if assist_data_raw else []
                    assist1 = assist_data[0] if len(assist_data) > 0 else None
                    assist2 = assist_data[1] if len(assist_data) > 1 else None

                    time = cols[5].text.strip()

                    scoring_event = {
                        'Period': period,
                        'Team': team,
                        'PP': pp,
                        'Player': player,
                        'Player_Goals': goals,
                        'Assist1': assist1,
                        'Assist2': assist2,
                        'Time': time
                    }
                    scoring_events.append(scoring_event)
                except Exception as e:
                    logging.error(f"An error occurred while parsing a scoring event row: {e}")
            else:
                logging.warning(f"Insufficient columns in scoring row: {len(cols)}")

    return pd.DataFrame(scoring_events)


############# PARSEING PENALTY SUMMARY WITH BS4

def parse_penalty_summary(html_content):
    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the penalties div and table
    penalties_div = soup.find('div', id='penalties')
    if penalties_div is None:
        logging.error("Penalties div not found")
        return None

    penalties_table = penalties_div.find('table')
    if penalties_table is None:
        logging.error("Penalties table not found within the penalties div")
        return None

    # Initialize list to store penalty events
    penalty_events = []
    period = None

    # Loop through table rows
    for row in penalties_table.find_all('tr'):
        if 'stats-section' in row.get('class', []):
            td = row.find('td')
            if td:
                period = td.text.strip()
            else:
                logging.warning("Period name not found in 'stats-section' row")
                period = "Unknown"
        else:
            cols = row.find_all('td')
            if len(cols) > 1:
                team = cols[0].text.strip()
                player = cols[1].text.strip()
                pen_length = cols[2].text.strip()
                penalty_type = cols[3].text.strip()
                time = cols[4].text.strip()

                penalty_event = {
                    'Period': period,
                    'Team': team,
                    'Player': player,
                    'Pen_Length': pen_length,
                    'Penalty_Type': penalty_type,
                    'Time': time
                }
                penalty_events.append(penalty_event)

    return pd.DataFrame(penalty_events)


############# GOALIE SUMMARY WITH BS4
def parse_goalie_stats(html_content):
    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the goalies div and table
    goalies_div = soup.find('div', id='goalies')
    if goalies_div is None:
        logging.error("Goalies div not found")
        return None

    goalies_table = goalies_div.find('table')
    if goalies_table is None:
        logging.error("Goalies table not found within the goalies div")
        return None

    # Initialize list to store goalie stats
    goalie_stats = []
    team = None

    # Loop through table rows
    for row in goalies_table.find_all('tr'):
        if 'stats-header' in row.get('class', []):
            td = row.find('td')
            if td:
                team = td.text.strip()
            else:
                logging.warning("Team name not found in 'stats-header' row")
                team = "Unknown"
        else:
            cols = row.find_all('td')
            if len(cols) > 1:
                goalie = cols[0].text.strip()
                sv = cols[1].text.strip()
                ga = cols[2].text.strip()
                minutes = cols[3].text.strip()

                goalie_stat = {
                    'Team': team,
                    'Goalie': goalie,
                    'SV': sv,
                    'GA': ga,
                    'Minutes': minutes
                }
                goalie_stats.append(goalie_stat)

    return pd.DataFrame(goalie_stats)



### Get the Linescore Elements - Score, shots, ect by period####
#DONE - I THINK ### NEEDS UPDATE NOW THAT POSTSEASON MEAND 5th, 6th, ect PERIODS

def parse_linescore(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    linescore_data = []
    
    # Parsing the Goals table
    goals_table = soup.select_one("#goals table")
    if goals_table is None:
        logging.error("Goals table not found")
        return None
    
    rows = goals_table.select('tbody tr')
    if not rows:
        logging.warning("No rows found in Goals table")
        return None
    
    for row in rows:
        team_data = {}
        td = row.select_one('td')
        if td:
            team_data['Team'] = td.text
        else:
            logging.warning("Team name not found in Goals table")
            continue

        goals = row.select('td')[1:]
        for i, goal in enumerate(goals):
            period = i + 1
            column_name = f'goals{period}' if i < len(goals) - 1 else 'goalsT'
            team_data[column_name] = int(goal.text)
        
        linescore_data.append(team_data)
    

    # Parsing the Shots table
    shots_table = soup.select_one("#shots table")
    if shots_table is None:
        logging.error("Shots table not found")
        return None

    rows = shots_table.select('tbody tr')
    if not rows:
        logging.warning("No rows found in Shots table")
        return None

    for i, row in enumerate(rows):
        shots = row.select('td')[1:]
        if not shots:
            logging.warning(f"No shot data found for row {i+1} in Shots table")
            continue

        for j, shot in enumerate(shots):
            period = j + 1
            column_name = f'shots{period}' if j < len(shots) - 1 else 'shotsT'
            try:
                linescore_data[i][column_name] = int(shot.text.strip())
            except ValueError:
                logging.warning(f"Could not convert shot data to integer for row {i+1}, column {j+1}")
                linescore_data[i][column_name] = None

    # Parsing the PP table
    pp_table = soup.select_one("#pp table")
    if pp_table is None:
        logging.error("PP table not found")
        return None

    rows = pp_table.select('tbody tr')
    if not rows:
        logging.warning("No rows found in PP table")
        return None

    for i, row in enumerate(rows):
        try:
            pen_pim = row.select('td')[1].text.split('‑')
            linescore_data[i]['Pen'] = int(pen_pim[0])
            linescore_data[i]['PIM'] = int(pen_pim[1])

            ppg_ppo = row.select('td')[2].text.split('‑')
            linescore_data[i]['PPG'] = int(ppg_ppo[0])
            linescore_data[i]['PPO'] = int(ppg_ppo[1])

            fow_fol = row.select('td')[3].text.split('‑')
            linescore_data[i]['FOW'] = int(fow_fol[0])
            linescore_data[i]['FOL'] = int(fow_fol[1])
            linescore_data[i]['FOW%'] = (linescore_data[i]['FOW'] / (linescore_data[i]['FOW'] + linescore_data[i]['FOL'])) * 100

        except (ValueError, IndexError) as e:
            logging.warning(f"Could not process PP data for row {i+1}. Error: {e}")
            continue

    # Convert to DataFrame early
    df = pd.DataFrame(linescore_data)

    # Ensure all columns exist
    expected_goals_columns = [f'goals{i}' for i in range(1, 7)] + ['goalsT']
    expected_shots_columns = [f'shots{i}' for i in range(1, 7)] + ['shotsT']

    for col in expected_goals_columns + expected_shots_columns:
        if col not in df.columns:
            df[col] = 0

    # Return the final DataFrame
    return df



# Function to parse game details
def parse_game_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    meta_div = soup.find('div', {'id': 'meta'})
    if meta_div is None:
        logging.error("Meta div not found")
        return None
    
    game_details_div = meta_div.find_all('div')[-1]
    if game_details_div is None:
        logging.error("Game details div not found")
        return None
    
    try:
        date_str = game_details_div.h4.string
        day_of_week, date = date_str.split(", ", 1)
        
        p_elements = game_details_div.find_all('p')
        
        # Extract conference and location details
        for p in p_elements:
            if "Game" in p.text:  # e.g., "Big Ten Game"
                details_strs = p.get_text(separator='|').split('|')
                conference = details_strs[0]
                location = details_strs[-1].split('at ')[-1]
                break
        else:  # Defaults if not found
            conference, location = None, None
        
        # Extract referees and assistant referees details
        for p in p_elements:
            if "Referees" in p.text:
                refs_str = p.strong.next_sibling if p.strong else None
                asst_refs_str = p.find_all('strong')[1].next_sibling if len(p.find_all('strong')) > 1 else None
                break
        else:  # Defaults if not found
            refs_str, asst_refs_str = None, None
        
        refs = refs_str.split(', ') if refs_str else []
        asst_refs = asst_refs_str.split(', ') if asst_refs_str else []
        refs = [re.sub(r'[^a-zA-Z ]+', '', ref).strip() for ref in refs]
        asst_refs = [re.sub(r'[^a-zA-Z ]+', '', ref).strip() for ref in asst_refs]
        
        # Extract attendance details using regex for better accuracy
        attendance_pattern = r"Attendance:\s?(\d+[\d,]*)"
        attendance_match = re.search(attendance_pattern, html_content)
        attendance = int(attendance_match.group(1).replace(',', '')) if attendance_match else None
        
        # Extract game details (like shootout results)
        details = None
        for p in p_elements:
            if "shootout" in p.text:
                details = p.text
                break
        
        # Clean details if present
        if details and '\n' in details:
            details = details.replace('\n', '').strip()
        if details and '\t' in details:
            details = re.sub('\t', ' ', details)
        
        game_details = {
            'Day': day_of_week,
            'Date': date,
            'Conference': conference,
            'Details': details,
            'Location': location,
            'Ref1': refs[0] if refs else None,
            'Ref2': refs[1] if len(refs) > 1 else None,
            'Asst_Ref1': asst_refs[0] if asst_refs else None,
            'Asst_Ref2': asst_refs[1] if len(asst_refs) > 1 else None,
            'Attendance': attendance
        }
        
        game_details_df = pd.DataFrame([game_details])
        return game_details_df

    except (AttributeError, IndexError, ValueError) as e:
        logging.error(f"Error while parsing game details: {e}")
        return None


# PARENT FUNCTION TO PARSE BOX SCORE HTML
def parse_box_score(box_score_html):
    # Initialize DataFrames to None
    scoring_summary = penalty_summary = goalie_stats = player_stats = linescore = game_details = None
    
    try:
        scoring_summary = parse_scoring_summary(box_score_html)
    except Exception as e:
        print(f"Error in parse_scoring_summary: {e}")
    
    try:
        penalty_summary = parse_penalty_summary(box_score_html)
    except Exception as e:
        print(f"Error in parse_penalty_summary: {e}")
    
    try:
        goalie_stats = parse_goalie_stats(box_score_html)
    except Exception as e:
        print(f"Error in parse_goalie_stats: {e}")
    
    try:
        player_stats = parse_player_summary(box_score_html)
    except Exception as e:
        print(f"Error in parse_player_summary: {e}")
    
    try:
        game_details = parse_game_details(box_score_html)
    except Exception as e:
        print(f"Error in parse_game_details: {e}")

    try: 
        linescore = parse_linescore(box_score_html)
    except Exception as e:
        print(f"Error in parse_linescore: {e}")
    
    # Combine DataFrames into a list
    all_dfs = [game_details, scoring_summary, penalty_summary, goalie_stats, player_stats, linescore]
    
    return all_dfs


def rename_duplicate_columns(df):
    cols = pd.Series(df.columns)
    for dup in df.columns[df.columns.duplicated()].unique(): 
        cols[df.columns.get_loc(dup)] = [f"{dup}_{i}" if i != 0 else dup for i in range(df.columns.get_loc(dup).sum())]
    df.columns = cols
    return df




# End

In [ ]:
# Define table names for these DataFrames
table_names = ['game_details', 'scoring_summary', 'penalty_summary', 
                'goalie_stats', 'player_stats', 'linescore']

In [ ]:
### Take Small Sample for Testing
# df = df.head(5)

# Perform The Scrape with a Loop

In [ ]:
########## NEW TEST WITH PROGRESS BAR

from tqdm import tqdm  # Import tqdm for progress bar

box_score_data = {}

# Initialize the progress bar
total_rows = len(df)
progress_bar = tqdm(total=total_rows, desc="Processing")

# Loop through each row of the DataFrame and parse the box score
for row in df.itertuples():
    # Get the box score link
    box_score_link = row.Box_Link
    
    game_id = row.Game_ID
    # Increment the progress bar
    # progress_bar.set_description(f"Processing box score for {game_id}: \n")
    
    # Get the box score page with requests
    box_score_response = requests.get(base_url + box_score_link)
    box_score_html = box_score_response.text
    # Parse the box score
    all_dfs = parse_box_score(box_score_html)
    # Store the DataFrames in the dictionary
    box_score_data[game_id] = dict(zip(table_names, all_dfs))
    
    # Increment the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

# Convert the dictionary to a list of dataframes
# DataFrames renamed to match the table names
for game_id, dfs in box_score_data.items():
    for table_name, df in dfs.items():
        if df is not None:
            # Add the game ID to the DataFrame
            df['Game_ID'] = game_id
            box_score_data[game_id][table_name] = df


# Save the DataFrames to a SQLite database
            
# Set file path and name
db_path = f'../TEMP/{FILE_TAG}.db'            
# Create a connection to the database
conn = sqlite3.connect(db_path)

# Add the tables to the database
for game_id, dfs in box_score_data.items():
    for table_name, df in dfs.items():
        if df is not None:
            # Rename duplicate columns
            # df = rename_duplicate_columns(df)
            # Save the DataFrame to the database
            df.to_sql(table_name, conn, if_exists='append', index=False)


In [ ]:
box_score_data

In [ ]:
all_dfs = [df for df in all_dfs if isinstance(df, pd.DataFrame)]

combined_df = pd.concat(all_dfs, axis=0)


In [ ]:
combined_df.info()

# Save to a CSV file
# combined_df.to_csv(f'../TEMP/{tag}_combined.csv', index=False)

# DB Cleaning and standarizing

In [ ]:

# Path to master roster file
roster_path = os.path.join(os.getcwd(), "..", "data", 'rosters', 'all_time_combined_roster.csv')

# Read in the master roster file
roster = pd.read_csv(roster_path)
# roster.head()

# Set Path to db file 
db_path = os.path.join(os.getcwd(), "..", 'TEMP', f'{season_year}_Box_Scores_v1.db')
# db_path = os.path.join(os.getcwd(), "..", 'data', 'db', '2018_Box_Scores.db')

# Connect to the database
conn = sqlite3.connect(db_path)
# Create a cursor
cur = conn.cursor()


# # Value Count of Season
# roster['Season'].value_counts()

# filter the roster to the season
roster_df = roster[roster['Season'] == season_year]

# roster_df.head()

## Create dictionary of Team Primary Names to Abbreviations

- Needed to Add IVY teams becuase of low amount of games. will have to do for harvard, yale, ect next week

In [ ]:
## Create dataframe from SQL query
df = pd.read_sql_query("SELECT * FROM scoring_summary", conn)

# Function to count the occurrences of primary team names for unmatched abbreviations
def count_primary_names_for_abbreviation(abbreviation):
    filtered_rows = df[df['Team'] == abbreviation]
    team_counts = {}
    
    for _, row in filtered_rows.iterrows():
        teams = row['Game_ID'].split('-')[-2:]
        for team in teams:
            if team not in team_counts:
                team_counts[team] = 0
            team_counts[team] += 1
            
    return team_counts


# Attempt to match abbreviations to primary names based on substrings and common naming conventions
matched_dict = {}
unmatched_abbreviations = []

for abbreviation in df['Team'].unique():



# Match the abbreviation to the primary team name with the highest occurrence

    team_counts = count_primary_names_for_abbreviation(abbreviation)
    # Get the team with the highest count
    matched_team = max(team_counts, key=team_counts.get)
    matched_dict[abbreviation] = matched_team

# matched_dict

# Manually fix the unmatched abbreviations - IVY League Teams with no of very few games throw a wrench in the above method
# Brown: Brown
# Cornell: Cornell

# Make those substitutions
matched_dict['Brown'] = 'Brown'
matched_dict['Cornell'] = 'Cornell'
# yale
matched_dict['Yale'] = 'Yale'
# princeton
matched_dict['Princeton'] = 'Princeton'

# harvard
matched_dict['Harvard'] = 'Harvard'
# columbia
matched_dict['Columbia'] = 'Columbia'

# dartmouth
matched_dict['Dartmouth'] = 'Dartmouth'

# penn
matched_dict['Penn'] = 'Penn'

# BC
matched_dict['BC'] = 'Boston College'



# matched_dict

## Add Home and Away Columns to game_details table

In [ ]:
# Step 1: Read the game_details table into a DataFrame
df_game_details = pd.read_sql("SELECT * FROM game_details", conn)

# Step 2: Create new columns for Home and Away Teams by parsing Game_ID
df_game_details['Away_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[3])
df_game_details['Home_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[4])

# Step 3: Write this updated DataFrame back to the game_details table
df_game_details.to_sql('game_details', conn, if_exists='replace', index=False)


## Clean up The Column Names and extra header rows in the Player Stats table

In [ ]:
############ 'Pt.' should be 'Pts' and '+/-' should be 'plus_minus'
#################################
player_stats_df = pd.read_sql_query("SELECT * FROM player_stats", conn)

# Define a dictionary for column renaming
column_renames = {
    'Pt.': 'Pts',
    '+/-': 'plus_minus'
}

# Rename columns based on the dictionary
player_stats_df.rename(columns=column_renames, inplace=True)


# Drop rows where Team name is in the Player column
player_stats_df = player_stats_df[player_stats_df['Team'] != player_stats_df['Player']]

## Change the Column names to be easy to work with
############ 'Pt.' should be 'Pts' and '+/-' should be 'plus_minus'
#################################
player_stats_df = pd.read_sql_query("SELECT * FROM player_stats", conn)

if 'Pt.' in player_stats_df.columns:
    player_stats_df.rename(columns={'Pt.': 'Pts'}, inplace=True)
else:
    print("Column 'Pt.' not found.")

if '+/-' in player_stats_df.columns:
    player_stats_df.rename(columns={'+/-': 'plus_minus'}, inplace=True)
else:
    print("Column '+/-' not found.")

print(len(player_stats_df))

# Drop rows if Team name is in the player column
# If ['Team'] is the same as ['Player'] then drop that row
player_stats_df = player_stats_df[player_stats_df['Team'] != player_stats_df['Player']]

# add the dataframe back to the database
player_stats_df.to_sql('player_stats', conn, if_exists='replace', index=False)

# print(len(player_stats_df))
#################################
# player_stats_df.head()

## Add The primary team names to the linescores table
# Read the linescores table into a DataFrame
df_linescores = pd.read_sql("SELECT * FROM linescore", conn)

# Apply the dictionary to the Team column
df_linescores['Team'] = df_linescores['Team'].apply(lambda x: matched_dict[x])

# df_linescores.head()


# Penalty Table & Scoring Summary

In [ ]:
## Add The primary team names to the linescores table
# Read the linescores table into a DataFrame

df_penalty = pd.read_sql("SELECT * FROM penalty_summary", conn)

# Apply the dictionary to the Team column
#$ Skip if not found

df_penalty['Team'] = df_penalty['Team'].apply(lambda x: matched_dict[x])

    

# Apply same method to scorring_summary:
df_scoring = pd.read_sql("SELECT * FROM scoring_summary", conn)
df_scoring['Team'] = df_scoring['Team'].apply(lambda x: matched_dict[x])

# Add Home and Away Team columns to scoring_summary
df_scoring['Away_Team'] = df_scoring['Game_ID'].apply(lambda x: x.split('-')[3])
df_scoring['Home_Team'] = df_scoring['Game_ID'].apply(lambda x: x.split('-')[4])


## Add each table back to database
# Write the updated linescores DataFrame back to the linescore table
df_linescores.to_sql('linescore', conn, if_exists='replace', index=False)

# Write the updated penalty DataFrame back to the penalty_summary table
df_penalty.to_sql('penalty_summary', conn, if_exists='replace', index=False)

# Write the updated scoring DataFrame back to the scoring_summary table
df_scoring.to_sql('scoring_summary', conn, if_exists='replace', index=False)

## CREATE A NEW TABLE WITH AGGRIGATED PLAYER STATS YEAR TO DATE

In [ ]:
# Use player_stats_df from here on, instead of running another SQL query.
df_player_stats = player_stats_df.copy()


# Clean up the name format in player_stats for easier matching
# Replace the non-breaking space with a regular space
df_player_stats['Clean_Player'] = df_player_stats['Player'].apply(lambda x: x.replace('\xa0', ' '))

# Remove rows where Player is the team name (e.g., "Michigan State")
df_player_stats_cleaned = df_player_stats[df_player_stats['Player'] != df_player_stats['Team']]

# Convert relevant columns to integers for correct aggregation
cols_to_convert = ['G', 'A', 'Pts', 'plus_minus', 'Sh', 'PIM']
for col in cols_to_convert:
    df_player_stats_cleaned[col] = pd.to_numeric(df_player_stats_cleaned[col], errors='coerce')

# Aggregate the data for year-to-date stats
# Add a column for counting the number of games each player has played
agg_player_stats_corrected_with_games = df_player_stats_cleaned.groupby(['Clean_Player', 'Team']).agg({
    'G': 'sum',
    'A': 'sum',
    'Pts': 'sum',
    'plus_minus': 'sum',
    'Sh': 'sum',
    'PIM': 'sum',
    'Game_ID': 'count'  # Counting the number of unique Game_IDs for each player
}).reset_index()

# Rename the Game_ID column to Games_Played
agg_player_stats_corrected_with_games.rename(columns={'Game_ID': 'Games_Played'}, inplace=True)

# Save the updated aggregated data back to the database, replacing the existing table
agg_player_stats_corrected_with_games.to_sql('player_stats_ytd', conn, if_exists='replace', index=False)

# Verify by loading some sample data from the updated table
sample_updated_ytd = pd.read_sql_query("SELECT * FROM player_stats_ytd LIMIT 5;", conn)
# sample_updated_ytd


## Add the Roster data to the Database

In [ ]:
################## SET THE ROSTER DATAFRAME TO THE CORRECT YEAR ####################
## MATCH THE DATAFRAME NAMES
df_master_roster = roster_df.copy()


# Clean up the name formats for joining
# Master roster: Convert "Last Name, First Name" to "First Name Last Name"
# df_master_roster['Clean_Name'] = df_master_roster['Player'].apply(lambda x: ' '.join(reversed(x.split(', '))))

# Rename Player to Clean_Name
roster_df.rename(columns={'Player': 'Clean_Name'}, inplace=True)
# Rename School to Team
roster_df.rename(columns={'School': 'Team'}, inplace=True)

# Clean up the Team column, remove '-' and replace with ' '
# df_master_roster['School'] = df_master_roster['Team'].apply(lambda x: x.replace('-', ' '))

## If there are an period in the column names, remove them
roster_df.columns = df_master_roster.columns.str.replace('.', '')

# Save the roster data as a new table in the database
roster_table_name = 'master_roster'
roster_df.to_sql(roster_table_name, conn, if_exists='replace', index=False)
############################################################

# Verify by listing all the tables in the database again
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = conn.execute(tables_query).fetchall()
table_names_updated = [table[0] for table in tables]
table_names_updated